# Performance Summary Usage 

**Performance Summary includes:**
- RAM usage
- FINE parameters
- TSAM parameters
- Gurobi values 

# Import Modules and Data

In [1]:
import fine as fn
import os

## Load Multi-regional Energy System Example

In [2]:
def getModel(data):
    cwd = os.getcwd()


    # %matplotlib inline  
    # %load_ext autoreload
    # %autoreload 2

    # %% [markdown]
    # # 2. Create an energy system model instance 
    #
    # The structure of the energy system model is given by the considered locations, commodities, the number of time steps as well as the hours per time step.
    #
    # The commodities are specified by a unit (i.e. 'GW_electric', 'GW_H2lowerHeatingValue', 'Mio. t CO2/h') which can be given as an energy or mass unit per hour. Furthermore, the cost unit and length unit are specified.

    # %%
    locations = {'cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6', 'cluster_7'}
    commodityUnitDict = {'electricity': r'GW$_{el}$', 'methane': r'GW$_{CH_{4},LHV}$', 'biogas': r'GW$_{biogas,LHV}$',
                        'CO2': r'Mio. t$_{CO_2}$/h', 'hydrogen': r'GW$_{H_{2},LHV}$'}
    commodities = {'electricity', 'hydrogen', 'methane', 'biogas', 'CO2'}
    numberOfTimeSteps=8760
    hoursPerTimeStep=1

    # %%
    esM = fn.EnergySystemModel(locations=locations, commodities=commodities, numberOfTimeSteps=8760,
                            commodityUnitsDict=commodityUnitDict,
                            hoursPerTimeStep=1, costUnit='1e9 Euro', lengthUnit='km', verboseLogLevel=0)

    # %%
    CO2_reductionTarget = 0.1

    # %% [markdown]
    # # 3. Add commodity sources to the energy system model

    # %% [markdown]
    # ## 3.1. Electricity sources

    # %% [markdown]
    # ### Wind onshore

    # %%
    esM.add(fn.Source(esM=esM, name='Wind (onshore)', commodity='electricity', hasCapacityVariable=True,
                    operationRateMax=data['Wind (onshore), operationRateMax'],
                    capacityMax=data['Wind (onshore), capacityMax'],
                    investPerCapacity=1.1, opexPerCapacity=1.1*0.02, interestRate=0.08,
                    economicLifetime=20))

    # %% [markdown]
    # Full load hours:

    # %%
    data['Wind (onshore), operationRateMax'].sum()

    # %% [markdown]
    # ### Wind offshore

    # %%
    esM.add(fn.Source(esM=esM, name='Wind (offshore)', commodity='electricity', hasCapacityVariable=True,
                    operationRateMax=data['Wind (offshore), operationRateMax'],
                    capacityMax=data['Wind (offshore), capacityMax'],
                    investPerCapacity=2.3, opexPerCapacity=2.3*0.02, interestRate=0.08,
                    economicLifetime=20))

    # %% [markdown]
    # Full load hours:

    # %%
    data['Wind (offshore), operationRateMax'].sum()

    # %% [markdown]
    # ### PV

    # %%
    esM.add(fn.Source(esM=esM, name='PV', commodity='electricity', hasCapacityVariable=True,
                    operationRateMax=data['PV, operationRateMax'], capacityMax=data['PV, capacityMax'],
                    investPerCapacity=0.65, opexPerCapacity=0.65*0.02, interestRate=0.08,
                    economicLifetime=25))

    # %% [markdown]
    # Full load hours:

    # %%
    data['PV, operationRateMax'].sum()

    # %% [markdown]
    # ### Exisisting run-of-river hydroelectricity plants

    # %%
    esM.add(fn.Source(esM=esM, name='Existing run-of-river plants', commodity='electricity',
                    hasCapacityVariable=True,
                    operationRateFix=data['Existing run-of-river plants, operationRateFix'], tsaWeight=0.01,
                    capacityFix=data['Existing run-of-river plants, capacityFix'],
                    investPerCapacity=0, opexPerCapacity=0.208))

    # %% [markdown]
    # ## 3.2. Methane (natural gas and biogas)

    # %% [markdown]
    # ### Natural gas

    # %%
    esM.add(fn.Source(esM=esM, name='Natural gas purchase', commodity='methane',
                    hasCapacityVariable=False, commodityCost=0.0331*1e-3))

    # %% [markdown]
    # ### Biogas

    # %%
    esM.add(fn.Source(esM=esM, name='Biogas purchase', commodity='biogas',
                    operationRateMax=data['Biogas, operationRateMax'], hasCapacityVariable=False,
                    commodityCost=0.05409*1e-3))

    # %% [markdown]
    # # 4. Add conversion components to the energy system model

    # %% [markdown]
    # ### Combined cycle gas turbine plants

    # %%
    esM.add(fn.Conversion(esM=esM, name='CCGT plants (methane)', physicalUnit=r'GW$_{el}$',
                        commodityConversionFactors={'electricity':1, 'methane':-1/0.6, 'CO2':201*1e-6/0.6},
                        hasCapacityVariable=True,
                        investPerCapacity=0.65, opexPerCapacity=0.021, interestRate=0.08,
                        economicLifetime=33))

    # %% [markdown]
    # ### New combined cycle gas turbine plants for biogas

    # %%
    esM.add(fn.Conversion(esM=esM, name='New CCGT plants (biogas)', physicalUnit=r'GW$_{el}$',
                        commodityConversionFactors={'electricity':1, 'biogas':-1/0.63},
                        hasCapacityVariable=True, 
                        investPerCapacity=0.7, opexPerCapacity=0.021, interestRate=0.08,
                        economicLifetime=33))

    # %% [markdown]
    # ### New combined cycly gas turbines for hydrogen

    # %%
    esM.add(fn.Conversion(esM=esM, name='New CCGT plants (hydrogen)', physicalUnit=r'GW$_{el}$',
                        commodityConversionFactors={'electricity':1, 'hydrogen':-1/0.63},
                        hasCapacityVariable=True, 
                        investPerCapacity=0.7, opexPerCapacity=0.021, interestRate=0.08,
                        economicLifetime=33))

    # %% [markdown]
    # ### Electrolyzers

    # %%
    esM.add(fn.Conversion(esM=esM, name='Electrolyzer', physicalUnit=r'GW$_{el}$',
                        commodityConversionFactors={'electricity':-1, 'hydrogen':0.7},
                        hasCapacityVariable=True, 
                        investPerCapacity=0.5, opexPerCapacity=0.5*0.025, interestRate=0.08,
                        economicLifetime=10))

    # %% [markdown]
    # ### rSOC

    # %%
    capexRSOC=1.5

    esM.add(fn.Conversion(esM=esM, name='rSOEC', physicalUnit=r'GW$_{el}$', linkedConversionCapacityID='rSOC',
                        commodityConversionFactors={'electricity':-1, 'hydrogen':0.6},
                        hasCapacityVariable=True, 
                        investPerCapacity=capexRSOC/2, opexPerCapacity=capexRSOC*0.02/2, interestRate=0.08,
                        economicLifetime=10))

    esM.add(fn.Conversion(esM=esM, name='rSOFC', physicalUnit=r'GW$_{el}$', linkedConversionCapacityID='rSOC',
                        commodityConversionFactors={'electricity':1, 'hydrogen':-1/0.6},
                        hasCapacityVariable=True, 
                        investPerCapacity=capexRSOC/2, opexPerCapacity=capexRSOC*0.02/2, interestRate=0.08,
                        economicLifetime=10))

    # %% [markdown]
    # # 5. Add commodity storages to the energy system model

    # %% [markdown]
    # ## 5.1. Electricity storage

    # %% [markdown]
    # ### Lithium ion batteries
    #
    # The self discharge of a lithium ion battery is here described as 3% per month. The self discharge per hours is obtained using the equation (1-$\text{selfDischarge}_\text{hour})^{30*24\text{h}} = 1-\text{selfDischarge}_\text{month}$.

    # %%
    esM.add(fn.Storage(esM=esM, name='Li-ion batteries', commodity='electricity',
                    hasCapacityVariable=True, chargeEfficiency=0.95,
                    cyclicLifetime=10000, dischargeEfficiency=0.95, selfDischarge=1-(1-0.03)**(1/(30*24)),
                    chargeRate=1, dischargeRate=1, doPreciseTsaModeling=False,
                    investPerCapacity=0.151, opexPerCapacity=0.002, interestRate=0.08,
                    economicLifetime=22))

    # %% [markdown]
    # ## 5.2. Hydrogen storage

    # %% [markdown]
    # ### Hydrogen filled salt caverns
    # The maximum capacity is here obtained by: dividing the given capacity (which is given for methane) by the lower heating value of methane and then multiplying it with the lower heating value of hydrogen.

    # %%
    esM.add(fn.Storage(esM=esM, name='Salt caverns (hydrogen)', commodity='hydrogen',
                    hasCapacityVariable=True, capacityVariableDomain='continuous',
                    capacityPerPlantUnit=133,
                    chargeRate=1/470.37, dischargeRate=1/470.37, sharedPotentialID='Existing salt caverns',
                    stateOfChargeMin=0.33, stateOfChargeMax=1, capacityMax=data['Salt caverns (hydrogen), capacityMax'],
                    investPerCapacity=0.00011, opexPerCapacity=0.00057, interestRate=0.08,
                    economicLifetime=30))

    # %% [markdown]
    # ## 5.3. Methane storage

    # %% [markdown]
    # ### Methane filled salt caverns

    # %%
    esM.add(fn.Storage(esM=esM, name='Salt caverns (biogas)', commodity='biogas',
                    hasCapacityVariable=True, capacityVariableDomain='continuous',
                    capacityPerPlantUnit=443,
                    chargeRate=1/470.37, dischargeRate=1/470.37, sharedPotentialID='Existing salt caverns',
                    stateOfChargeMin=0.33, stateOfChargeMax=1, capacityMax=data['Salt caverns (methane), capacityMax'],
                    investPerCapacity=0.00004, opexPerCapacity=0.00001, interestRate=0.08,
                    economicLifetime=30))

    # %% [markdown]
    # ## 5.4 Pumped hydro storage

    # %% [markdown]
    # ### Pumped hydro storage

    # %%
    esM.add(fn.Storage(esM=esM, name='Pumped hydro storage', commodity='electricity',
                    chargeEfficiency=0.88, dischargeEfficiency=0.88,
                    hasCapacityVariable=True, selfDischarge=1-(1-0.00375)**(1/(30*24)),
                    chargeRate=0.16, dischargeRate=0.12, capacityFix=data['Pumped hydro storage, capacityFix'],
                    investPerCapacity=0, opexPerCapacity=0.000153))

    # %% [markdown]
    # # 6. Add commodity transmission components to the energy system model

    # %% [markdown]
    # ## 6.1. Electricity transmission

    # %% [markdown]
    # ### AC cables

    # %% [markdown]
    # esM.add(fn.LinearOptimalPowerFlow(esM=esM, name='AC cables', commodity='electricity',
    #                                   hasCapacityVariable=True, capacityFix=data['AC cables, capacityFix'],
    #                                   reactances=data['AC cables, reactances']))

    # %%
    esM.add(fn.Transmission(esM=esM, name='AC cables', commodity='electricity',
                                    hasCapacityVariable=True, capacityFix=data['AC cables, capacityFix']))

    # %% [markdown]
    # ### DC cables

    # %%
    esM.add(fn.Transmission(esM=esM, name='DC cables', commodity='electricity', losses=data['DC cables, losses'],
                            distances=data['DC cables, distances'],
                            hasCapacityVariable=True, capacityFix=data['DC cables, capacityFix']))

    # %% [markdown]
    # ## 6.2 Methane transmission

    # %% [markdown]
    # ### Methane pipeline

    # %%
    esM.add(fn.Transmission(esM=esM, name='Pipelines (biogas)', commodity='biogas', 
                            distances=data['Pipelines, distances'],
                            hasCapacityVariable=True, hasIsBuiltBinaryVariable=True, bigM=300,
                            locationalEligibility=data['Pipelines, eligibility'],
                            capacityMax=data['Pipelines, eligibility']*15, sharedPotentialID='pipelines',
                            investPerCapacity=0.000037, investIfBuilt=0.000314,
                            interestRate=0.08, economicLifetime=40))

    # %% [markdown]
    # ## 6.3 Hydrogen transmission

    # %% [markdown]
    # ### Hydrogen pipelines

    # %%
    esM.add(fn.Transmission(esM=esM, name='Pipelines (hydrogen)', commodity='hydrogen',
                            distances=data['Pipelines, distances'],
                            hasCapacityVariable=True, hasIsBuiltBinaryVariable=True, bigM=300,
                            locationalEligibility=data['Pipelines, eligibility'],
                            capacityMax=data['Pipelines, eligibility']*15, sharedPotentialID='pipelines',
                            investPerCapacity=0.000177, investIfBuilt=0.00033,
                            interestRate=0.08, economicLifetime=40))

    # %% [markdown]
    # # 7. Add commodity sinks to the energy system model

    # %% [markdown]
    # ## 7.1. Electricity sinks

    # %% [markdown]
    # ### Electricity demand

    # %%
    esM.add(fn.Sink(esM=esM, name='Electricity demand', commodity='electricity',
                    hasCapacityVariable=False, operationRateFix=data['Electricity demand, operationRateFix']))

    # %% [markdown]
    # ## 7.2. Hydrogen sinks

    # %% [markdown]
    # ### Fuel cell electric vehicle (FCEV) demand

    # %%
    FCEV_penetration=0.5
    esM.add(fn.Sink(esM=esM, name='Hydrogen demand', commodity='hydrogen', hasCapacityVariable=False,
                    operationRateFix=data['Hydrogen demand, operationRateFix']*FCEV_penetration))

    # %% [markdown]
    # ## 7.3. CO2 sinks

    # %% [markdown]
    # ### CO2 exiting the system's boundary

    # %%
    esM.add(fn.Sink(esM=esM, name='CO2 to enviroment', commodity='CO2',
                    hasCapacityVariable=False, commodityLimitID='CO2 limit', yearlyLimit=366*(1-CO2_reductionTarget)))




    return esM


In [3]:
from getData import getData
data = getData()
esM = getModel(data)

The distances of a component are set to a normalized value of 1.


# Run Model with Performance Summary

__Description__

The function esM.optimizeAndIncludePerformanceSummary is a wrapper around esM.optimize and additionally stores a performance summary (in Dataframe format) as attribute ('esM.performanceSummary') in the esM instance.

__Usage__

Replace *esM.optimize(...)* with *esM.optimizeAndIncludePerformanceSummary(...)* to get a performance summary after optimization. Arguments do not have to be changed.

__Notes__

- the following packages are required: psutil, grblogtools

- if TSA is used, *storeTSAinstance* should be set to True. Only then the TSA parameters are saved in the ESM instance
- a log file name should be specified. logFileName='xyz.log' This log file is used by Gurobi to store the log and later used to extract relevant Gurobi parameters


In [ ]:
# Usage

logFileName = 'run.log'
# If time series aggregation is enabled, the TSA instance needs to be saved (storeTSAinstance=True) in order to be included in the performance summary
esM.aggregateTemporally(numberOfTypicalPeriods=2,storeTSAinstance=True)

# with performance summary 
esM.optimize(
    timeSeriesAggregation=True,
    logFileName=logFileName, 
    optimizationSpecs='OptimalityTol=1e-3 method=2 cuts=0 MIPGap=5e-3',
    includePerformanceSummary=True)


Clustering time series data with 2 typical periods and 24 time steps per period...
		(0.7781 sec)

Time series aggregation specifications:
Number of typical periods:2, number of time steps per period:24

Declaring sets, variables and constraints for SourceSinkModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.1040 sec)

Declaring sets, variables and constraints for ConversionModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.0329 sec)

Declaring sets, variables and constraints for StorageModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(1.1790 sec)

Declaring sets, variables and constraints for TransmissionModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.0736 sec)

Declaring shared potential constraint...
		(0.0016 sec)

Declaring linked component quantity constraint...
		(0.0000 sec)

Declaring commodity balances...
		(0.1296 sec)

		(0.0002 sec)

Declaring

In [5]:
# Access the performance Summary
import pandas as pd
pd.set_option('display.max_rows', 500)
esM.performanceSummary

Value
Category        Parameter                                                                   
FineParameters  noOfRegions                                                                8
                numberOfTimeSteps                                                       8760
                hoursPerTimestep                                                           1
                numberOfYears                                                            1.0
                optimizationSpecs             OptimalityTol=1e-3 method=2 cuts=0 MIPGap=5e-3
RAMUsage        ramUsageStartGB                                                      0.35651
                ramUsageEndGB                                                       0.418644
ProcessingTimes buildtime                                                           1.580103
                tsaBuildTime                                                        0.778096
                solvetime                                                           3.859111
                runtime                                                             7.203635
TSAParameters   clusterMethod                                                   hierarchical
                noTypicalPeriods                                                           2
                hoursPerPeriod                                                            24
                segmentation                                                           False
                noSegments                                                                10
                tsaSolver                                                                cbc
                timeStepsPerPeriod                                                        24
                tsaBuildTime                                                        0.778096
GurobiSummary   Version                                                                9.5.1
                BarIterCount                                                              41
                NodeCount                                                                  1
                IterCount                                                               4514
                Runtime                                                                 1.66
                RelaxObj                                                            39.42376
                RelaxIterCount                                                          4514
                RelaxTime                                                               1.38
                SolCount                                                                   1
                OrderingTime                                                            0.12
                Threads                                                                    3
                Cores                                                                     72
                PresolveTime                                                            0.17
                NumConstrs                                                             44417
                NumVars                                                                22723
                NumNZs                                                                130996
                ObjVal                                                             39.472327
                PresolvedNumConVars                                                    14210
                PresolvedNumIntVars                                                       26
                PresolvedNumBinVars                                                       26
                ObjBound                                                           39.423765
                MIPGap                                                               0.00123
                Platform                                                             lin